In [52]:
import pandas as pd
import numpy as np
import datetime as dt
import os 

In [53]:
time_series = pd.read_csv("../csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
time_series.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20
0,16.0,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316.0,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,141,141,141,141,145,145,145,145,145,145
2,580.0,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,14,14,14,14,14,14,14,14,14,14
3,630.0,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,1371,1389,1400,1433,1539,1575,1757,1808,1843,1924
4,850.0,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,57,57,57,57,66,66,66,66,66,66


In [54]:
time_series = time_series.dropna(subset = ["Admin2"])
time_series.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20
5,84001001.0,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,37,39,40,43,44,42,45,48,53,53
6,84001003.0,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,161,168,171,174,174,175,181,187,188,189
7,84001005.0,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,33,35,37,37,39,42,43,45,45,47
8,84001007.0,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,38,42,42,42,42,42,42,43,42,43
9,84001009.0,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,34,34,34,36,37,39,40,40,40,40


In [55]:
sites = pd.read_excel("../clean_data/sites.xlsx")
sites.head()

,Site,Region,County,State,Combined_Key
0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US"
1,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US"


In [56]:
time_series_sites = time_series.merge(sites, left_on = "Combined_Key", right_on = "Combined_Key", how = "inner")
time_series_sites.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,Site,Region,County,State
0,84055061.0,US,USA,840,55061.0,Kewaunee,Wisconsin,US,44.517275,-87.614684,...,14,13,15,18,22,22,Point Beach,3,Kewaunee,Wisconsin
1,84055071.0,US,USA,840,55071.0,Manitowoc,Wisconsin,US,44.119468,-87.809289,...,11,11,13,13,14,16,Point Beach,3,Manitowoc,Wisconsin


In [57]:
#time_series_sites = time_series.merge(sites, left_on = "Combined_Key", right_on = "Combined_Key", how = "left")
#time_series_sites.head()

In [58]:
#time_series_sites = time_series_sites.dropna(subset = ["Site"])
#time_series_sites.head()

In [59]:
counties_sites = time_series_sites[["FIPS", "Site", "Region", "County","State","Combined_Key"]]
counties_sites.head()

,FIPS,Site,Region,County,State,Combined_Key
0,55061.0,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US"
1,55071.0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US"


In [60]:
dates = time_series_sites.iloc[:,68:116]
dates.head()

,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,...,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20
0,0,0,0,0,0,0,0,0,0,0,...,10,10,11,14,14,13,15,18,22,22
1,0,0,0,0,0,0,0,0,0,0,...,10,11,11,11,11,11,13,13,14,16


In [61]:
county_time_series = pd.DataFrame(columns = ["FIPS","Site","Region","County","State","Combined_Key","Confirmed"])

In [62]:
#county_time = pd.DataFrame(columns = ["Confirmed"])
cols = dates.columns

for i in range(0,len(cols)-1):
    one_day = pd.DataFrame(columns = ["Confirmed", "Date"])
    one_day["Confirmed"] = dates.iloc[:,i]
    one_day["Date"] = cols[i]
    county_time = counties_sites.join(one_day)
    county_time_series = county_time_series.append(county_time)
#    county_time.append(one_day)
#    print(one_day)
county_time_series.head()


,FIPS,Site,Region,County,State,Combined_Key,Confirmed,Date
0,55061.0,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US",0,3/19/20
1,55071.0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",0,3/19/20
0,55061.0,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US",0,3/20/20
1,55071.0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",0,3/20/20
0,55061.0,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US",0,3/21/20


In [63]:
county_time_series.reset_index(inplace = True)
county_time_series.drop("index", axis = 1, inplace = True)

In [64]:
county_time_series["Date"] = pd.to_datetime(county_time_series.Date)
county_time_series.dtypes

FIPS                   float64
Site                    object
Region                  object
County                  object
State                   object
Combined_Key            object
Confirmed               object
Date            datetime64[ns]
dtype: object

In [65]:
county_time_series["Date"] = county_time_series["Date"].dt.to_period("D")

In [66]:
county_time_series.tail()

,FIPS,Site,Region,County,State,Combined_Key,Confirmed,Date
89,55071.0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",13,2020-05-02
90,55061.0,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US",18,2020-05-03
91,55071.0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",13,2020-05-03
92,55061.0,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US",22,2020-05-04
93,55071.0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",14,2020-05-04


In [67]:
county_time_series.to_excel("../clean_data/site_time_series.xlsx")